In [1]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imutils import paths
from keras.models import Sequential
from keras.applications.xception import Xception
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
import argparse
import imutils
import ntpath
import cv2
import os

import talos as ta
import wrangle as wr
from talos.metrics.keras_metrics import fmeasure_acc
from talos import live

import pandas as pd

from keras.optimizers import Adam, Nadam, Adadelta
from keras.activations import relu, elu
from keras.losses import categorical_crossentropy

%matplotlib inline

log_device_placement = True

Using TensorFlow backend.


In [2]:
cd /home/carlywolfbrandt/galvanize/assignments/csk_image_recognition/images/

/home/carlywolfbrandt/galvanize/assignments/csk_image_recognition/images


In [64]:
for subdir, dirs, files in os.walk('/home/carlywolfbrandt/galvanize/assignments/csk_image_recognition/images/initial_dataset/train'):
    print(dirs)

['64016024', '53744727', '478708', '24177852', '5857493', '494155', '5863037', '1064995', '313791339', '36091883', '535078', '48650376', '497397', '516411', '615713']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [7]:
ls

initial_dataset/  model_images/  molecular_images/


In [23]:
datagen = ImageDataGenerator(featurewise_std_normalization = True,
                                    featurewise_center = True,
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    rescale=1./255,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

for subdir, dirs, files in os.walk('/home/carlywolfbrandt/galvanize/assignments/csk_image_recognition/images/initial_dataset/validation_4'):
    for dr in dirs:
        i = 0
        img = load_img(f'/home/carlywolfbrandt/galvanize/assignments/csk_image_recognition/images/initial_dataset/train_4/{dr}/{dr}.png')
        x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
        x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)
        for batch in datagen.flow(x, batch_size=10,
                              save_to_dir=f'/home/carlywolfbrandt/galvanize/assignments/csk_image_recognition/images/initial_dataset/train_4/{dr}/', save_format='png'):
            i += 1
            print(i)
            if i > 1:
                break 

1
2
1
2
1
2
1
2


In [ ]:
# dimensions of our images.
img_width, img_height = 50, 50


train_data_dir = 'molecular_images'
validation_data_dir = 'molecular_images'
nb_train_samples = 1458
nb_validation_samples = 1458
epochs = 250
batch_size = 25
nb_filters = 32
extra_filters = 32
pool_size = (2, 2)
kernel_size = (3, 3)
act = 'relu'
opt = Adam(lr=0.001, decay=1e-6)


if K.image_data_format() == 'channels_first':
    input_shape = (3 , img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(nb_filters, kernel_size, input_shape=input_shape))
model.add(Activation(act))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Conv2D(nb_filters, kernel_size))
model.add(Activation(act))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Conv2D(nb_filters + extra_filters, kernel_size))
model.add(Activation(act))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Flatten())
model.add(Dense(nb_filters + extra_filters))
model.add(Activation(act))
model.add(Dropout(0.1))

model.add(Dense(nb_train_samples))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
               metrics=['accuracy'])

# model.load_weights('/home/carlywolfbrandt/galvanize/assignments/csk_image_recognition/models/model_10.h5')
model.load_weights('/home/carlywolfbrandt/galvanize/assignments/csk_image_recognition/models/model_relu_250_001.h5')

train_datagen = ImageDataGenerator(featurewise_std_normalization = True,
                                    featurewise_center = True,
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    rescale=1./255,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# model = Xception(weights=None, input_shape=(img_height,img_width,3), classes=15)
# model.compile(loss='categorical_crossentropy',
#               optimizer='Adam',
#                metrics=['accuracy'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=1)

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

model.save_weights('/home/carlywolfbrandt/galvanize/assignments/csk_image_recognition/models/model_relu_250_001.h5')

Found 1458 images belonging to 1458 classes.
Found 1458 images belonging to 1458 classes.
Epoch 1/250
58/58 [==============================] - 3s 56ms/step - loss: 2.9947 - acc: 0.3531 - val_loss: 0.7770 - val_acc: 0.7600
Epoch 2/250
58/58 [==============================] - 2s 33ms/step - loss: 2.5624 - acc: 0.4113 - val_loss: 0.6331 - val_acc: 0.8000
Epoch 3/250
58/58 [==============================] - 2s 36ms/step - loss: 2.5315 - acc: 0.4120 - val_loss: 0.9617 - val_acc: 0.7200
Epoch 4/250
58/58 [==============================] - 2s 36ms/step - loss: 2.3650 - acc: 0.4271 - val_loss: 0.5706 - val_acc: 0.9200
Epoch 5/250
58/58 [==============================] - 2s 37ms/step - loss: 2.1626 - acc: 0.4583 - val_loss: 0.4663 - val_acc: 0.8400
Epoch 6/250
58/58 [==============================] - 2s 38ms/step - loss: 2.2269 - acc: 0.4430 - val_loss: 0.6376 - val_acc: 0.8400
Epoch 7/250
58/58 [==============================] - 2s 37ms/step - loss: 2.1435 - acc: 0.4742 - val_loss: 0.3868 - va

Epoch 62/250
58/58 [==============================] - 2s 33ms/step - loss: 1.5626 - acc: 0.5590 - val_loss: 1.1418 - val_acc: 0.5600
Epoch 63/250
58/58 [==============================] - 2s 33ms/step - loss: 1.4761 - acc: 0.5887 - val_loss: 1.1315 - val_acc: 0.6000
Epoch 64/250
58/58 [==============================] - 2s 32ms/step - loss: 1.6694 - acc: 0.5349 - val_loss: 1.2235 - val_acc: 0.5600
Epoch 65/250
58/58 [==============================] - 2s 33ms/step - loss: 1.5269 - acc: 0.5602 - val_loss: 0.6673 - val_acc: 0.8400
Epoch 66/250
58/58 [==============================] - 2s 34ms/step - loss: 1.5775 - acc: 0.5541 - val_loss: 1.0355 - val_acc: 0.8800
Epoch 67/250
14/58 [======>.......................] - ETA: 1s - loss: 1.5122 - acc: 0.5671

In [25]:
# dimensions of our images.
img_width, img_height = 50, 50


train_data_dir = 'molecular_images'
validation_data_dir = 'molecular_images'
nb_train_samples = 1458
nb_validation_samples = 1458
epochs = 250
batch_size = 25
nb_filters = 32
extra_filters = 32
pool_size = (2, 2)
kernel_size = (3, 3)
act = 'elu'
opt = Adam(lr=0.001, decay=1e-6)


if K.image_data_format() == 'channels_first':
    input_shape = (3 , img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(nb_filters, kernel_size, input_shape=input_shape))
model.add(Activation(act))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Conv2D(nb_filters, kernel_size))
model.add(Activation(act))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Conv2D(nb_filters + extra_filters, kernel_size))
model.add(Activation(act))
model.add(MaxPooling2D(pool_size=pool_size))

model.add(Flatten())
model.add(Dense(nb_filters + extra_filters))
model.add(Activation(act))
model.add(Dropout(0.1))

model.add(Dense(nb_train_samples))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
               metrics=['accuracy'])

model.load_weights('/home/carlywolfbrandt/galvanize/assignments/csk_image_recognition/models/model_relu_250_001.h5')

train_datagen = ImageDataGenerator(featurewise_std_normalization = True,
                                    featurewise_center = True,
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    rescale=1./255,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

history = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=1)

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

model.save_weights('/home/carlywolfbrandt/galvanize/assignments/csk_image_recognition/models/model_fullchange.h5')

Found 1458 images belonging to 1458 classes.
Found 1458 images belonging to 1458 classes.
Epoch 1/100
58/58 [==============================] - 3s 50ms/step - loss: 1.7817 - acc: 0.4883 - val_loss: 3.7471 - val_acc: 0.3200
Epoch 2/100
58/58 [==============================] - 2s 35ms/step - loss: 1.4571 - acc: 0.5810 - val_loss: 3.7014 - val_acc: 0.4000
Epoch 3/100
58/58 [==============================] - 2s 36ms/step - loss: 1.4854 - acc: 0.5715 - val_loss: 5.7199 - val_acc: 0.2000
Epoch 4/100
58/58 [==============================] - 2s 36ms/step - loss: 1.4338 - acc: 0.5702 - val_loss: 4.2034 - val_acc: 0.3200
Epoch 5/100
58/58 [==============================] - 2s 37ms/step - loss: 1.5163 - acc: 0.5334 - val_loss: 3.7660 - val_acc: 0.3600
Epoch 6/100
58/58 [==============================] - 2s 38ms/step - loss: 1.4758 - acc: 0.5432 - val_loss: 4.0513 - val_acc: 0.3600
Epoch 7/100
58/58 [==============================] - 2s 37ms/step - loss: 1.5527 - acc: 0.5438 - val_loss: 5.9171 - va

KeyboardInterrupt: 